In [62]:
import sys
sys.path.append('..')
import constants.file_handler_constants as fh
from packages.attraction.Attraction import *
from packages.file_handler_package.file_handler import *
import os
import glob
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv, dotenv_values 
import google.generativeai as genai
from google.generativeai.types import ContentType
from PIL import Image
from IPython.display import Markdown
import time

import json

In [52]:
createDirectory(os.curdir, 'temp')
# loading variables from .env file
load_dotenv() 

# URL of the image
all_img_url = ['https://lh5.googleusercontent.com/p/AF1QipP1FKj5WaMEZjBw--nU9NRM0fb8y_Dc9gAyINMG=s483-k-no', 
               'https://lh5.googleusercontent.com/p/AF1QipPHdiEQzV1-UTx2uw3ClV3y9DYdk6cG-npwpfY=s846-k-no', 
               'https://lh5.googleusercontent.com/p/AF1QipO5-fmJEGgV--VHolyOp8zy6C4s1tqNL_9oSMW2=s508-k-no', 
               'https://lh5.googleusercontent.com/p/AF1QipNzhoLzYcGI1_BIsp6QpimaUmo-XhJ2ZAEUak9p=s644-k-no', 
               'https://lh5.googleusercontent.com/p/AF1QipMmUEYH_-Fe9NdfZQfF02txCgftNQJC-d9ony7v=s846-k-no', 'https://lh5.googleusercontent.com/p/AF1QipNxPsgNP1Dqj2Hdb5ldxotfmYkOKr1btZ_24sdW=s641-k-no', 'https://lh5.googleusercontent.com/p/AF1QipPz8FjBrcGDx6CEgjJVnFTB7zEFALOqoCYYIeyn=s563-k-no', 'https://lh5.googleusercontent.com/p/AF1QipPCf2no1tlEOcy7opR_S_kudr-uN-9Fa-57IpeJ=s483-k-no', 'https://lh5.googleusercontent.com/p/AF1QipM5TNd8WZKZSbG_lpZaopHV9zF7FL7RxWz4Kony=s635-k-no', 'https://lh5.googleusercontent.com/p/AF1QipPiRPVpvFaJ6FV3rWY_gxGGz6nhHXrlZdRVvk_l=s544-k-no', 'https://lh5.googleusercontent.com/p/AF1QipPZzYEWBwERCvbsFVBZKjCBQDc6mrF4HmnFdFEJ=s508-k-no', 'https://lh5.googleusercontent.com/p/AF1QipOHSYh7CkUgPpXAyO9MKYclp25iQVL_SenfLiRd=s483-k-no', 'https://lh5.googleusercontent.com/p/AF1QipN7P2epvcchZ1Pb10ISYRTIywnBVG-nMpPIc-Ak=s563-k-no', 'https://lh5.googleusercontent.com/p/AF1QipNp-rE6cd7laHQEdTHjIWJQMBhscydxgMXoRare=s483-k-no', 'https://lh5.googleusercontent.com/p/AF1QipM0_HRR4EVarkObtPRAyQjuH5XvI4hzMBxIGU1R=s644-k-no', 'https://lh5.googleusercontent.com/p/AF1QipOGtXxzBaPC85Yq4ifR6MlezRGlgTxIWnwmo1pN=s483-k-no', 'https://lh5.googleusercontent.com/p/AF1QipPTKzNu-yLhQGvtJaDfouU6M_iDKb9g9WDE2mVe=s483-k-no', 'https://lh5.googleusercontent.com/p/AF1QipNUHkdiEppwzFp6SeUUF11V7v3Iw0n31q7L9Lfx=s483-k-no', 'https://lh5.googleusercontent.com/p/AF1QipMUQhWBXdww1hRM-GNvSfbZ40vvgFT-0v0rmkr2=s483-k-no', 'https://lh5.googleusercontent.com/p/AF1QipN61ebAcAA32E4NQMPcgYC22bQTQIFyMSAFfBIt=s508-k-no', 
               'https://lh5.googleusercontent.com/p/AF1QipMNPU5onZHTbpkyxYb7C2yjpoAW9xXweM9PSzHF=s623-k-no']


for cur_url in all_img_url:
    response = requests.get(cur_url)
    if response.status_code == 200:
        filename = 'temp/{0}.jpeg'.format(cur_url.split('/')[-1])
        with open(filename, 'wb') as file:
            file.write(response.content)


## send api to get score for current attraction (send with query and main image)
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
model = genai.GenerativeModel('gemini-1.5-pro-latest')
text_prompt = "Provide place name, latitude, and longitude. I will return a JSON string containing scores (0-1) for following attributes(nothing else no other sentences)" + \
"\nfor example: \'{\"Tourism\":0,\"Adventure\":0,\"Meditation\":0,\"Art\":0,\"Cultural\":0,\"Landscape\":0,\"Nature\":0,\"Historical\":0,\"Cityscape\":0,\"Beach\":0,\"Mountain\":0,\"Architecture\":0,\"Temple\":0,\"WalkingStreet\":0,\"Market\":0,\"Village\":0,\"NationalPark\":0,\"Diving\":0,\"Snuggle\":0,\"Waterfall\":0,\"Island\":0,\"Shopping\":0,\"Camping\":0,\"Fog\":0,\"Cycling\":0,\"Monument\":0,\"Zoo\":0,\"Waterpark\":0,\"Hiking\":0,\"Museum\":0,\"Riverside\":0,\"NightLife\":0,\"Family\":0,\"Kid\":0,\"Landmark\":0,\"Forest\":0}" + \
"\nตลาดชุมชนริมคลองบางหลวง, 13.7319158, 100.4629207 give me score for this"

# prompt the model
prompt = [text_prompt]
for cur_path_img in glob.glob(os.path.join(r'C:\Users\user\git\CEPPWebScraping\test_workspace\temp', '*.jpeg')):
    cur_img_prompt = Image.open(cur_path_img)
    prompt.append(cur_img_prompt)
response = model.generate_content(prompt)

# remove directory 'temp'
removeNoneEmptyDir('temp')

res_start_Idx = response.text.find('{')
res_end_Idx = response.text.find('}')
res_score = response.text[res_start_Idx:res_end_Idx+1]
print(res_score)

Directory temp created successfully
{"Tourism": 0.8, "Adventure": 0.2, "Meditation": 0.1, "Art": 0.3, "Cultural": 0.9, "Landscape": 0.4, "Nature": 0.3, "Historical": 0.7, "Cityscape": 0.1, "Beach": 0, "Mountain": 0, "Architecture": 0.2, "Temple": 0.6, "WalkingStreet": 0.1, "Market": 0.7, "Village": 0.9, "NationalPark": 0, "Diving": 0, "Snuggle": 0, "Waterfall": 0, "Island": 0, "Shopping": 0.2, "Camping": 0, "Fog": 0, "Cycling": 0.1, "Monument": 0, "Zoo": 0, "Waterpark": 0, "Hiking": 0, "Museum": 0.1, "Riverside": 1.0, "NightLife": 0, "Family": 0.5, "Kid": 0.2, "Landmark": 0.2, "Forest": 0}


In [68]:
test = "Provide place name, latitude, and longitude. I will return a JSON string containing scores (0-1) for following attributes(nothing else no other sentences)" + \
"\nfor example: \'{\"Tourism\":0,\"Adventure\":0,\"Meditation\":0,\"Art\":0,\"Cultural\":0,\"Landscape\":0,\"Nature\":0,\"Historical\":0,\"Cityscape\":0,\"Beach\":0,\"Mountain\":0,\"Architecture\":0,\"Temple\":0,\"WalkingStreet\":0,\"Market\":0,\"Village\":0,\"NationalPark\":0,\"Diving\":0,\"Snuggle\":0,\"Waterfall\":0,\"Island\":0,\"Shopping\":0,\"Camping\":0,\"Fog\":0,\"Cycling\":0,\"Monument\":0,\"Zoo\":0,\"Waterpark\":0,\"Hiking\":0,\"Museum\":0,\"Riverside\":0,\"NightLife\":0,\"Family\":0,\"Kid\":0,\"Landmark\":0,\"Forest\":0}" + \
"\nSiam Paragon, 13.7464265, 100.5322802 give me score for this"
print(test)
x_str = '{"Tourism": 0.44, "Adventure":0, "Meditation": 0,"Art": 0}'
x_json = json.loads(x_str)
x_json['Tourism']

Provide place name, latitude, and longitude. I will return a JSON string containing scores (0-1) for following attributes(nothing else no other sentences)
for example: '{"Tourism":0,"Adventure":0,"Meditation":0,"Art":0,"Cultural":0,"Landscape":0,"Nature":0,"Historical":0,"Cityscape":0,"Beach":0,"Mountain":0,"Architecture":0,"Temple":0,"WalkingStreet":0,"Market":0,"Village":0,"NationalPark":0,"Diving":0,"Snuggle":0,"Waterfall":0,"Island":0,"Shopping":0,"Camping":0,"Fog":0,"Cycling":0,"Monument":0,"Zoo":0,"Waterpark":0,"Hiking":0,"Museum":0,"Riverside":0,"NightLife":0,"Family":0,"Kid":0,"Landmark":0,"Forest":0}
Siam Paragon, 13.7464265, 100.5322802 give me score for this


0.44

In [54]:
print('\'')

'


In [55]:
print("for example: \'{\"Tourism\":0,\"Camping\":0,\"Kid\":0,\"Landmark\":0,\"Forest\":0}'")

for example: '{"Tourism":0,"Camping":0,"Kid":0,"Landmark":0,"Forest":0}'


In [56]:
x = 'https://lh5.googleusercontent.com/p/AF1QipP1FKj5WaMEZjBw--nU9NRM0fb8y_Dc9gAyINMG=s483-k-no'
print(x.split('/'))

['https:', '', 'lh5.googleusercontent.com', 'p', 'AF1QipP1FKj5WaMEZjBw--nU9NRM0fb8y_Dc9gAyINMG=s483-k-no']
